In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import requests

In [2]:
driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')

/var/folders/tj/9f4bs_0n69b_w28_xyk24sfh0000gn/T/ipykernel_1735/1949518271.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')


In [3]:
driver.get('https://www.amazon.in/')

In [4]:
val = input('Enter product here:-')

Enter product here:-guitar


In [5]:
search_bar = driver.find_element("xpath",'//*[@id="twotabsearchtextbox"]')    # Finding the search bar using it's xpath
search_bar.send_keys(val)       # Inputing keyword to search 
search_button = driver.find_element("xpath",'//*[@id="nav-search-submit-button"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [6]:
productName=[]

In [7]:
#scraping the Product_Name 
PName=driver.find_elements("xpath","//span[@class='a-size-medium a-color-base a-text-normal']")
for i in PName:
    if i.text is None :
        productName.append("--") 
    else:
        productName.append(i.text)
print(len(productName),productName)

2 ['Kadence Guitar Acoustica Series, 41” Jumbo Size 6-strings Acoustic Guitar, Spruce body with Pick Guard A311 with bag… B07JKYWGJN (KAD-A-311)', 'Kadence Guitar Acoustica Series, 41” Jumbo Size 6-strings Acoustic Guitar, Spruce body with Pick Guard A311 with bag… B07JKYWGJN (KAD-A-311)']


# Q2

In [8]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements("xpath",'//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
       # nxt_button = driver.find_element("xpath",'//li[@class="a-last"]/a')      # Locating the next_button which is active
       # if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
           # nxt_button.click()                                                    # Clicking the next button
           # time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        #elif driver.find_element("xpath",'//li[@class="a-disabled a-last"]/a').text == 'Next→':    
          #  print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
          #  break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [9]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [10]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element("xpath",'//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element("xpath",'//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element("xpath",'//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element("xpath",'//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element("xpath",'//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element("xpath",'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element("xpath",'//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element("xpath",'//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element("xpath",'//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4NjEyODc1MjQ0Njg3MTgyOjE2NzMwNDQ1NTI6c3BfYXRmOjIwMDYxNjIwNjk2MjA0OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dguitar%26qid%3D1673044552%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1%26smid%3DAM04Z7MH4HSDD
Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4NjEyODc1MjQ0Njg3MTgyOjE2NzMwNDQ1NTI6c3BfYXRmOjIwMTAyODQxMTY0OTk4OjowOjo&url=%2FMedellin-Acoustic-Guitar-Carbon-Learning%2Fdp%2FB09MSGMQ3Y%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dguitar%26qid%3D1673044552%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1%26smid%3DAM04Z7MH4HSDD
Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4NjEyODc1MjQ0Njg3MTgyOjE2NzMwNDQ1NTI6c3BfYXRmOjIwMDU3MTkzMDU2MjA0OjowOjo&url=%2FKadence-Slowhand-Premium-Acoustic-Instrument%2Fdp%2FB077SZ667X%2Fref%3Dsr_1_3_sspa%3Fkeywords%3Dguitar%26qid%3D1673044552%26sr%3D8-3-spons%26sp_csd%3Dd2lkZ2V0TmFt

In [11]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,Visit the Kadence Store,Kadence Frontier guitar with Online Guitar lea...,4.0 out of 5 stars,"1,859 ratings",-,7 days Replacement,"Monday, Jan 9",,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Brand: Medellin,Medellin M38 carbon fiber body 38 Incheses Aco...,3.8 out of 5 stars,181 ratings,-,7 days Replacement,-,In stock.,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the Kadence Store,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4.0 out of 5 stars,553 ratings,-,7 days Replacement,"Thursday, Jan 12",,ACOUSTIC-ELECTRIC GUITAR: Kadence Slowhand sem...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Brand: Medellin,"Medellin 38"" Acoustic Guitar premium wood with...",5.0 out of 5 stars,1 rating,-,7 days Replacement,-,In stock.,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [12]:
#saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(input))

# Q3

In [13]:
driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')

/var/folders/tj/9f4bs_0n69b_w28_xyk24sfh0000gn/T/ipykernel_1735/1949518271.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')


In [14]:
driver.get('https://images.google.com/')

In [15]:
search_bar = driver.find_element("xpath",'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element("xpath",'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button


In [16]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page...


In [17]:
images = driver.find_elements("xpath",'//img[@class="rg_i Q4LuWd"]')

In [18]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

192

In [19]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("/Users/raj/Desktop/banana/img"+str(i)+".jpg", "wb")
    file.write(response.content)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/raj/Desktop/banana/img0.jpg'

#  Q4

In [20]:
driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')
url4="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url4)

/var/folders/tj/9f4bs_0n69b_w28_xyk24sfh0000gn/T/ipykernel_1735/2815685439.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')


In [21]:
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [22]:
BName=driver.find_elements("xpath","//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

24 ['REDMI Note 11 SE (Cosmic White, 64 GB)', 'MOTOROLA e40 (Carbon Gray, 64 GB)', 'REDMI 9i Sport (Coral Green, 64 GB)', 'POCO C31 (Shadow Gray, 64 GB)', 'POCO C31 (Royal Blue, 64 GB)', 'REDMI 9i Sport (Carbon Black, 64 GB)', 'MOTOROLA G32 (Mineral Gray, 64 GB)', 'REDMI 9i Sport (Metallic Blue, 64 GB)', 'MOTOROLA e40 (Pink Clay, 64 GB)', 'MOTOROLA G32 (Satin Silver, 64 GB)', 'MOTOROLA g52 (Metallic White, 128 GB)', 'MOTOROLA G62 5G (Midnight Gray, 128 GB)', 'APPLE iPhone 14 (Starlight, 128 GB)', 'APPLE iPhone 14 (Midnight, 128 GB)', 'MOTOROLA g52 (Charcoal Grey, 128 GB)', 'APPLE iPhone 14 (Purple, 128 GB)', 'REDMI 10 (Midnight Black, 64 GB)', 'POCO M4 Pro 5G (Cool Blue, 128 GB)', 'APPLE iPhone 13 (Starlight, 128 GB)', 'MOTOROLA G62 5G (Frosted Blue, 128 GB)', 'POCO X4 Pro 5G (Laser Blue, 128 GB)', 'APPLE iPhone 14 (Blue, 128 GB)', 'REDMI 10 (Caribbean Green, 64 GB)', 'Infinix HOT 20 Play (Luna Blue, 64 GB)']


In [23]:
#scraping the Storage_RAM_ROM 
ram=driver.find_elements("xpath","//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['6 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM', '6 GB RAM | 128 GB ROM', '6 GB RAM | 128 GB ROM', '128 GB ROM', '128 GB ROM', '6 GB RAM | 128 GB ROM', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '128 GB ROM', '6 GB RAM | 128 GB ROM', '6 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 256 GB']


In [24]:
#scraping the P_F_Camera 
PC=driver.find_elements("xpath","//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
print(len(P_F_Camera),P_F_Camera)

24 ['64MP + 8MP + 2MP + 2MP | 13MP Front Camera', '48MP + 2MP + 2MP | 8MP Front Camera', '13MP Rear Camera | 5MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '13MP Rear Camera | 5MP Front Camera', '50MP + 8MP + 2MP | 16MP Front Camera', '13MP Rear Camera | 5MP Front Camera', '48MP + 2MP + 2MP | 8MP Front Camera', '50MP + 8MP + 2MP | 16MP Front Camera', '50MP + 8MP + 2MP | 16MP Front Camera', '50MP + 8MP + 2MP | 16MP Front Camera', '12MP + 12MP | 12MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP + 8MP + 2MP | 16MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP + 2MP | 5MP Front Camera', '50MP + 8MP | 16MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP + 8MP + 2MP | 16MP Front Camera', '64MP + 8MP + 2MP | 16MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP + 2MP | 5MP Front Camera', '13Mp + AI Lens | 8MP Front Camera']


In [25]:
#scraping the Display_size_Resolution 
DS=driver.find_elements("xpath","//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)

24 ['16.33 cm (6.43 inch) Full HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '16.64 cm (6.55 inch) Full HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.64 cm (6.55 inch) Full HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.64 cm (6.55 inch) Full HD+ Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '16.76 cm (6.6 inch) Full HD+ Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '17.02 cm (6.7 inch) HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '16.64 cm (6.55 inch) Full HD+ Display', '16.94 cm (6.67 inch) Full HD+ Super AMOLED Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '17.02 cm (6.7 inch) HD+ Display', '17.32 cm (6.82 inch) HD+ Display']


In [26]:
#scraping the ProcessorAndCores 
P=driver.find_elements("xpath","//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
print(len(ProcessorAndCores),ProcessorAndCores)

24 ['Mediatek Helio G95 Octa Core Processor', 'UNISOC T700 Processor', 'MediaTek Helio G25 Processor', 'MediaTek Helio G35 Processor', 'MediaTek G35 Processor', 'MediaTek Helio G25 Processor', 'Qualcomm Snapdragon 680 Processor', 'MediaTek Helio G25 Processor', 'UNISOC T700 Processor', 'Qualcomm Snapdragon 680 Processor', 'Qualcomm Snapdragon 680 Processor', 'Qualcomm Snapdragon 695 5G Processor', '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories', '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories', 'Qualcomm Snapdragon 680 Processor', '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories', 'Qualcomm Snapdragon 680 Processor', 'Mediatek Dimensity 810 Processor', 'Brand Warranty for 1 Year', 'Qualcomm Snapdragon 695 5G Processor', 'Qualcomm Snapdragon 695 5G Processor', '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories', 'Qualcomm Snapdragon 680 Processor', 'MediaTek G37 Processor']


In [27]:
#scraping the Battery 
B=driver.find_elements("xpath","//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)

24 ['5000 mAh Lithium Polymer Battery', '5000 mAh Battery', '5000 mAh Li-Polymer Battery', '5000 mAh Lithium-ion Polymer Battery', '5000 mAh Lithium-ion Polymer Battery', '5000 mAh Li-Polymer Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Li-Polymer Battery', '5000 mAh Battery', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium Battery', '5000 mAh Lithium Polymer Battery', 'A15 Bionic Chip, 6 Core Processor Processor', 'A15 Bionic Chip, 6 Core Processor Processor', '5000 mAh Lithium Battery', 'A15 Bionic Chip, 6 Core Processor Processor', '6000 mAh Lithium Polymer Battery', '5000 mAh Lithium-ion Polymer Battery', 'A15 Bionic Chip Processor', '5000 mAh Lithium Polymer Battery', '5000 mAh Lithium-ion Polymer Battery', 'A15 Bionic Chip, 6 Core Processor Processor', '6000 mAh Lithium Polymer Battery', '6000 mAh Li-ion Polymer Battery']


In [28]:
#scraping the Price 
price=driver.find_elements("xpath","//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)

24 ['₹11,499', '₹7,999', '₹7,199', '₹7,499', '₹7,499', '₹7,199', '₹9,999', '₹7,199', '₹7,999', '₹9,999', '₹12,999', '₹14,999', '₹73,990', '₹73,990', '₹12,999', '₹73,990', '₹9,499', '₹14,999', '₹61,999', '₹14,999', '₹16,999', '₹73,990', '₹9,499', '₹8,499']


In [29]:
FlipKart=pd.DataFrame([])
FlipKart['Brand_Name']=Brand_Name
FlipKart['Storage_RAM_ROM']=Storage_RAM_ROM
FlipKart['Amount P_F_Camera']=P_F_Camera
FlipKart['Display_size_Resolution']=Display_size_Resolution
FlipKart['ProcessorAndCores']=ProcessorAndCores
FlipKart['Battery']=Battery
FlipKart['Price']=Price

FlipKart

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"REDMI Note 11 SE (Cosmic White, 64 GB)",6 GB RAM | 64 GB ROM | Expandable Upto 512 GB,64MP + 8MP + 2MP + 2MP | 13MP Front Camera,16.33 cm (6.43 inch) Full HD+ Display,Mediatek Helio G95 Octa Core Processor,5000 mAh Lithium Polymer Battery,"₹11,499"
1,"MOTOROLA e40 (Carbon Gray, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,48MP + 2MP + 2MP | 8MP Front Camera,16.51 cm (6.5 inch) HD+ Display,UNISOC T700 Processor,5000 mAh Battery,"₹7,999"
2,"REDMI 9i Sport (Coral Green, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G25 Processor,5000 mAh Li-Polymer Battery,"₹7,199"
3,"POCO C31 (Shadow Gray, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G35 Processor,5000 mAh Lithium-ion Polymer Battery,"₹7,499"
4,"POCO C31 (Royal Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek G35 Processor,5000 mAh Lithium-ion Polymer Battery,"₹7,499"
5,"REDMI 9i Sport (Carbon Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G25 Processor,5000 mAh Li-Polymer Battery,"₹7,199"
6,"MOTOROLA G32 (Mineral Gray, 64 GB)",4 GB RAM | 64 GB ROM,50MP + 8MP + 2MP | 16MP Front Camera,16.64 cm (6.55 inch) Full HD+ Display,Qualcomm Snapdragon 680 Processor,5000 mAh Lithium Polymer Battery,"₹9,999"
7,"REDMI 9i Sport (Metallic Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP Rear Camera | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,MediaTek Helio G25 Processor,5000 mAh Li-Polymer Battery,"₹7,199"
8,"MOTOROLA e40 (Pink Clay, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,48MP + 2MP + 2MP | 8MP Front Camera,16.51 cm (6.5 inch) HD+ Display,UNISOC T700 Processor,5000 mAh Battery,"₹7,999"
9,"MOTOROLA G32 (Satin Silver, 64 GB)",4 GB RAM | 64 GB ROM,50MP + 8MP + 2MP | 16MP Front Camera,16.64 cm (6.55 inch) Full HD+ Display,Qualcomm Snapdragon 680 Processor,5000 mAh Lithium Polymer Battery,"₹9,999"


#  Q5

In [30]:
driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')

/var/folders/tj/9f4bs_0n69b_w28_xyk24sfh0000gn/T/ipykernel_1735/1949518271.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')


In [36]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element('id','searchboxinput')                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element('id',"searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

current_url=driver.current_url
print('Current url :',current_url)

try:
    if "@" in current_url:
        location=current_url.split('@')[1].split(',*/data')[0].split(',')
        location
        print('Latitude of given Location:',location[0])
        print('Longitude of given Location:',location[1])
except:
    print('Location detail not found in url')

Enter City Name : banglore
Current url : https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@12.9544595,77.3012709,10z/data=!3m1!4b1!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627
Latitude of given Location: 12.9544595
Longitude of given Location: 77.3012709


# Q7

In [37]:
driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')

/var/folders/tj/9f4bs_0n69b_w28_xyk24sfh0000gn/T/ipykernel_1735/1949518271.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome('//Users/raj/Desktop/chromedriver')


In [38]:
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [39]:
driver.get(url)

In [40]:
Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [41]:
br=driver.find_elements("xpath","//div[@class='TopNumbeHeading']")
len(br)

10

In [42]:
for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

['1.MSI Titan GT77-12UHS',
 '2.Alienware X17 R2',
 '3.Acer Predator Triton 500 SE PT516-52s',
 '4.Omen By HP (16-B1371TX)',
 '5.Acer Predator Helios 300 AN515-45',
 '6.MSI Delta 15 (A5EFK-083IN)',
 '7.Omen By HP (16-C0141AX)',
 '8.Lenovo Legion 5i Pro (82RF00MGIN)',
 '9.Alienware M15 R5 Ryzen Edition Icc-C780001win',
 '10.Lenovo Slim 7 Gen 6 (82K8002JIN)']

In [43]:
sp=driver.find_elements("xpath","//div[@class='Specs-Wrap']")
len(sp)

10

In [44]:
for i in sp:
   
    Specification.append(str(i.text).replace("\n",""))
Specification

['Windows 11 Home OS17.3" (3840 x 2160) Display12th Gen Intel Core i9-12900HX | NA Processor2 TB SSD/64 GB DDR5 Memory',
 'Windows 11 Home OS17.3" (1920 x 1080) Display12th Gen Intel Core i9-12900H | 5 GHz Processor1 TB SSD/32 GB DDR5 Memory',
 'Windows 11 OS16" (2560 x 1600) Display12th Gen Intel Core i7-12700H | 3.5 GHz Processor2 TB SSD/32 GB DDR5 Memory',
 'Windows 11 Home OS16.1" (2560 x 1440) Display12th Gen Intel Core i7-12700H | 4.7 GHz Processor1 TB SSD/8 GB DDR5 Memory',
 'Windows 11 Home OS15.6" (2560 x 1440) DisplayAMD Ryzen 9-5900HX | NA Processor512 GB SSD/16 GB DDR4 Memory',
 'Windows 11 Home OS15.6" (1920 x 1080) DisplayAMD 5th Gen Ryzen 9-5900HX | 3.3GHz Processor1 TB SSD/16 GBGB DDR4 Memory',
 'Windows 11 Home OS16.1" (2560 x 1440) DisplayAMD Ryzen™ 9 5900HX | 3.3 GHz Processor1 TB NVMe/16 GB DDR4 Memory',
 'Windows 11 Home OS16" (2560 x 1600) Display12th Gen Intel Core i7-12700H | 2.3 GHz Processor1 TB SSD/16 GB DDR5 Memory',
 'Windows 11 Home OS15.6" (1920 x 1080) D

In [45]:
des=driver.find_elements("xpath","//div[@class='Section-center']")
len(des)

10

In [46]:
for i in des:
   
    Products_Description.append(str(i.text).replace("\n",""))
Products_Description

['The very best MSI gaming laptop featuring an Intel processor is the MSI TITAN GT77 - 12UHS. This is the top of the line model from MSI’s recent refresh of their entire gaming lineup and is powered by the Intel Core i9-12900HX which is a 16 Core (8P+8E) processor that has 24 threads and a maximum Turbo power of 157 Watts. The graphics card in this laptop is the NVIDIA RTX 3080 Ti and it too sports a high TGP of 175 Watts. This is one of the most powerful CPU+GPU combinations with plenty of thermal headroom to allow it to operate at higher clock speeds for longer time periods. As for the memory, the MSI GT77-12UHS can have up to 128 GB of DDR5 memory but ships with 64 GB DDR5 clocked at 4800 MT/s. Being a 12th Gen machine also means that the motherboard has PCIe Gen 5.0 lanes for storage devices such as M.2 SSDs. The GT77-12UHS has four M.2 slots of which one slot is PCIe Gen 5 whereas the remaining three are PCIe Gen 4 slots. Then there’s also the display, the GT77-12UHS features a ma

In [47]:
pri=driver.find_elements("xpath","//td[@class='smprice']")
len(pri)

10

In [48]:
for i in pri:
   
    Price.append(str(i.text).replace("\n",""))
Price

['N/A',
 '₹ 389,990',
 '₹ 229,990',
 '₹ 176,490',
 '₹ 172,999',
 '₹ 188,990',
 '₹ 124,990',
 '₹ 169,900',
 '₹ 174,000',
 '₹ 131,990']

In [49]:
digit_lap=pd.DataFrame([])
digit_lap['Brands']=Brands[0:10]
digit_lap['Price']=Price[0:10]
digit_lap['Specification']=Specification[0:10]
digit_lap['Description']=Products_Description[0:10]
digit_lap

,Brands,Price,Specification,Description
0,1.MSI Titan GT77-12UHS,N/A,"Windows 11 Home OS17.3"" (3840 x 2160) Display1...",The very best MSI gaming laptop featuring an I...
1,2.Alienware X17 R2,"₹ 389,990","Windows 11 Home OS17.3"" (1920 x 1080) Display1...","Alienware, the premium gaming brand from Dell ..."
2,3.Acer Predator Triton 500 SE PT516-52s,"₹ 229,990","Windows 11 OS16"" (2560 x 1600) Display12th Gen...",The Predator lineup from Acer has shown great ...
3,4.Omen By HP (16-B1371TX),"₹ 176,490","Windows 11 Home OS16.1"" (2560 x 1440) Display1...",Omen by HP is a premium gaming lineup which is...
4,5.Acer Predator Helios 300 AN515-45,"₹ 172,999","Windows 11 Home OS15.6"" (2560 x 1440) DisplayA...",Acer Predator Helios 300 AN515-45 is the best ...
5,6.MSI Delta 15 (A5EFK-083IN),"₹ 188,990","Windows 11 Home OS15.6"" (1920 x 1080) DisplayA...",The best MSI gaming laptop with an AMD process...
6,7.Omen By HP (16-C0141AX),"₹ 124,990","Windows 11 Home OS16.1"" (2560 x 1440) DisplayA...",Another worthy gaming laptop from the Omen lin...
7,8.Lenovo Legion 5i Pro (82RF00MGIN),"₹ 169,900","Windows 11 Home OS16"" (2560 x 1600) Display12t...",Lenovo has gone a little mainstream with their...
8,9.Alienware M15 R5 Ryzen Edition Icc-C780001win,"₹ 174,000","Windows 11 Home OS15.6"" (1920 x 1080) DisplayA...",Dell not only has gaming laptops under the Del...
9,10.Lenovo Slim 7 Gen 6 (82K8002JIN),"₹ 131,990",Windows 11 Home OS15.6 MP | NA DisplayLenovo R...,Lenovo has been pushing for thinner gaming lap...
